### Setting 

Install pandas QBQ

In [ ]:
!pip install pandas-gbq

  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Not uninstalling google-auth-oauthlib at /shared-libs/python3.9/py/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'google-auth-oauthlib'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires google-auth-oauthlib<0.5,>=0.4.1, but you have google-auth-oauthlib 1.2.0 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


Importing libraries

In [ ]:
import requests
import pandas as pd
import json
import base64
from urllib.parse import urlencode
from google.oauth2 import service_account
import os
import re
from pandas import json_normalize
from pandas.io.json import json_normalize

### Extract

Setting client id and client secrect 

In [ ]:
#Creating Client ID variable
CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")

#Creating Client Secret variable
CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

#Declaring Access Token variable
ACCESS_TOKEN = '' # I need to request to Spotify EP

Encoding client id and client secret to base64

In [ ]:
#Concatenating client id and client secret with the format required by spotify
client_creds = f"{CLIENT_ID}:{CLIENT_SECRET}"

#Enconding
client_creds_b64 = base64.b64encode(client_creds.encode())

Requesting my token

In [ ]:
#This is the ep to request the token
token_ep = 'https://accounts.spotify.com/api/token'

#Creating params
token_params = {'grant_type':'client_credentials'}

#Creating headers
token_headers = {'Authorization': f'Basic {client_creds_b64.decode()}'}

#Requesting and saving response in a variable
token_response = requests.post(token_ep,data=token_params,headers=token_headers)

#Getting access token 
ACCESS_TOKEN = token_response.json()['access_token']

Creating main header

In [ ]:
#This variable will be used in all requests
HEADERS = {'Authorization': f'Bearer {ACCESS_TOKEN}'}

Typing my artist

In [ ]:
search_input = 'Mac Miller'

Requesting information about my artist

In [ ]:
#First I clean the input
search_input = str(search_input.replace(' ','+'))

#This is the ep to request a search
search_ep = 'https://api.spotify.com/v1/search'

#Here I need to send params it is important to select the type o search and market
search_params = {'q':f"{search_input}",'type':'artist','market':'MX'}

#Requesting information
search = requests.get(search_ep,headers=HEADERS,params=search_params)

#Saving response
search = search.json()

Pointing to search with the text we want to search returns the results sorted by popularity, so we could access the first result to get the artist we are looking for.

In [ ]:
#Print response
search['artists']['items'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4LLpKhyESsyAXpc4laK94U'},
 'followers': {'href': None, 'total': 9936389},
 'genres': ['hip hop', 'pittsburgh rap', 'rap'],
 'href': 'https://api.spotify.com/v1/artists/4LLpKhyESsyAXpc4laK94U',
 'id': '4LLpKhyESsyAXpc4laK94U',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebed3b89aa602145fde71a163a',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174ed3b89aa602145fde71a163a',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178ed3b89aa602145fde71a163a',
   'width': 160}],
 'name': 'Mac Miller',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:4LLpKhyESsyAXpc4laK94U'}

Getting artist ID

In [ ]:
artist_id = search['artists']['items'][0]['id']

Getting all albums from my artist

In [ ]:
#This is the ep for get album data
albums_ep = f'https://api.spotify.com/v1/artists/{artist_id}/albums'

#I pass the params
albums_params = {'country':'MX','limit':'50'}

#Saving response
albums_response = requests.get(albums_ep,headers=HEADERS,params=albums_params).json()

#Creating a df with the response
pd.DataFrame(albums_response['items']).sample(5)

,album_group,album_type,artists,external_urls,href,id,images,is_playable,name,release_date,release_date_precision,total_tracks,type,uri
33,single,single,[{'external_urls': {'spotify': 'https://open.s...,{'spotify': 'https://open.spotify.com/album/2I...,https://api.spotify.com/v1/albums/2INwmkuiQbfi...,2INwmkuiQbfikxmwwLZwBY,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,Lucky Ass Bitch (feat. Juicy J),2013-03-26,day,1,album,spotify:album:2INwmkuiQbfikxmwwLZwBY
9,album,album,[{'external_urls': {'spotify': 'https://open.s...,{'spotify': 'https://open.spotify.com/album/1a...,https://api.spotify.com/v1/albums/1aDhas2rvJmi...,1aDhas2rvJmiGw3zVVI3mm,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,Live From Space,2013-12-17,day,14,album,spotify:album:1aDhas2rvJmiGw3zVVI3mm
15,album,album,[{'external_urls': {'spotify': 'https://open.s...,{'spotify': 'https://open.spotify.com/album/1j...,https://api.spotify.com/v1/albums/1jzqEyjugAp9...,1jzqEyjugAp9iLtRsj9LZg,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,K.I.D.S.,2010-08-13,day,16,album,spotify:album:1jzqEyjugAp9iLtRsj9LZg
2,album,album,[{'external_urls': {'spotify': 'https://open.s...,{'spotify': 'https://open.spotify.com/album/5S...,https://api.spotify.com/v1/albums/5SKnXCvB4fcG...,5SKnXCvB4fcGSZu32o3LRY,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,Faces,2021-10-15,day,25,album,spotify:album:5SKnXCvB4fcGSZu32o3LRY
26,single,single,[{'external_urls': {'spotify': 'https://open.s...,{'spotify': 'https://open.spotify.com/album/7G...,https://api.spotify.com/v1/albums/7GrWzoQHbx46...,7GrWzoQHbx463gP1E6HFSp,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",True,Buttons,2018-05-29,day,1,album,spotify:album:7GrWzoQHbx463gP1E6HFSp


Saving album ids

In [ ]:
#Saving al album ids in a list
albums = [album['id'] for album in albums_response['items']]

Getting all tracks for each album

In [ ]:
#This list will contain all the track ids
tracks_id = list()

#Create a loop for extract every track id for every album
for album in albums:
    tracks_ep = f'https://api.spotify.com/v1/albums/{album}/tracks'
    tracks_params = {'market':'MX'}
    tracks_response = requests.get(tracks_ep,headers=HEADERS,params=tracks_params).json()
    for track in tracks_response['items']:
        tracks_id.append(track['id'])

Getting all information about the tracks and creating the master dataframe

In [ ]:
#This is the row data for every track
raw_data = []

#Getting all track information for each track of the artist
for id in tracks_id:
    try:
        track_info_ep = f'https://api.spotify.com/v1/tracks/{id}'
        track_info_params = {'market':'MX'}
        track_info_response = requests.get(track_info_ep,headers=HEADERS,params=track_info_params).json()
        if 'error' in track_info_response:
            pass
        else:
            raw_data.append(track_info_response)
    except Exception as e:
        print(e)
        pass

In [ ]:
df = pd.DataFrame(raw_data)
df.sample(5)

,album,artists,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,is_playable,name,popularity,preview_url,track_number,type,uri
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,196756,True,{'isrc': 'USQY51374452'},{'spotify': 'https://open.spotify.com/track/3P...,https://api.spotify.com/v1/tracks/3P0mZWUWThSo...,3P0mZWUWThSofCgvC9ZJYH,False,True,Avian,50,https://p.scdn.co/mp3-preview/3669a44cf5e774ba...,2,track,spotify:track:3P0mZWUWThSofCgvC9ZJYH
310,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,168863,True,{'isrc': 'USAT22106841'},{'spotify': 'https://open.spotify.com/track/26...,https://api.spotify.com/v1/tracks/26PSkNzsEBx1...,26PSkNzsEBx1BRRB4vY4n5,False,True,Insure My Wrist (with Gunna),47,https://p.scdn.co/mp3-preview/e448a4f8fc887958...,10,track,spotify:track:26PSkNzsEBx1BRRB4vY4n5
238,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,186238,True,{'isrc': 'USA2P2015958'},{'spotify': 'https://open.spotify.com/track/27...,https://api.spotify.com/v1/tracks/27gJm5GVMhse...,27gJm5GVMhseR4daRo8UVQ,False,True,Get Em Up,42,https://p.scdn.co/mp3-preview/f7f09639b80ac602...,3,track,spotify:track:27gJm5GVMhseR4daRo8UVQ
44,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,223794,True,{'isrc': 'USWB12103321'},{'spotify': 'https://open.spotify.com/track/2C...,https://api.spotify.com/v1/tracks/2CNaYt7KbFqU...,2CNaYt7KbFqUrATbZZPY21,False,True,Funeral,55,https://p.scdn.co/mp3-preview/2393d3391a998bf1...,12,track,spotify:track:2CNaYt7KbFqUrATbZZPY21
380,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,1,193093,True,{'isrc': 'USQX91600320'},{'spotify': 'https://open.spotify.com/track/2N...,https://api.spotify.com/v1/tracks/2Nh2cMryoXl7...,2Nh2cMryoXl7BrZoIeN2Pr,False,True,Dapper (feat. Anderson .Paak),56,https://p.scdn.co/mp3-preview/bc35e40cee1f733c...,9,track,spotify:track:2Nh2cMryoXl7BrZoIeN2Pr


### Transform

In [ ]:
#album_data = json_normalize(df['album'])
#album_data.sample(5)

In [ ]:
df['featured_artist'] = df['artists'].apply(lambda row: [artist['name'] for artist in row])
#pd.DataFrame(df['featured_artist'].values.tolist()).add_prefix('artist_') 
df['featured_artist'] = df['featured_artist'].apply(lambda row: ', '.join(row[1:]))

In [ ]:
df['album_id'] = df['album'].apply(lambda x: x['id'])
df['album_name'] = df['album'].apply(lambda x: x['name'])
df['album_release_date'] = df['album'].apply(lambda x: x['release_date'])  
df['album_cover'] = df['album'].apply(lambda x: x['images'][0]['url'])
df['album_type'] = df['album'].apply(lambda x: x['album_type'])
df['album_total_tracks'] = df['album'].apply(lambda x: x['total_tracks'])  

In [ ]:
df['artist_id'] = df['artists'].apply(lambda x: x[0]['id']) 
df['artist_name'] = df['artists'].apply(lambda x: x[0]['name']) 

In [ ]:
df.columns

Index(['album', 'artists', 'disc_number', 'duration_ms', 'explicit',
       'external_ids', 'external_urls', 'href', 'id', 'is_local',
       'is_playable', 'name', 'popularity', 'preview_url', 'track_number',
       'type', 'uri', 'featured_artist', 'album_id', 'album_name',
       'album_release_date', 'album_cover', 'album_type', 'album_total_tracks',
       'artist_id', 'artist_name'],
      dtype='object')

In [ ]:
df = df[[
    'id',
    'name',
    'duration_ms',
    'popularity',
    'track_number',
    'album_id',
    'album_name',
    'album_release_date',
    'album_total_tracks',
    'album_cover',
    'album_type',
    'artist_id',
    'artist_name',
    'featured_artist'
]]

In [ ]:
df.sample(5)

,id,name,duration_ms,popularity,track_number,album_id,album_name,album_release_date,album_total_tracks,album_cover,album_type,artist_id,artist_name,featured_artist
238,27gJm5GVMhseR4daRo8UVQ,Get Em Up,186238,42,3,1jzqEyjugAp9iLtRsj9LZg,K.I.D.S.,2010-08-13,16,https://i.scdn.co/image/ab67616d0000b2739d37f9...,album,4LLpKhyESsyAXpc4laK94U,Mac Miller,
235,3l5r1zd9EWibbvOhBM8sQR,Back In The Day,246865,52,18,6TFm9KGW7Lbxrl2Zsg4X9R,K.I.D.S. (Deluxe),2010-08-13,18,https://i.scdn.co/image/ab67616d0000b273463d73...,album,4LLpKhyESsyAXpc4laK94U,Mac Miller,
160,2R8dXjdGLvOQAqYgpQXBtl,Suplexes Inside of Complexes and Duplexes,167437,45,12,5lftuHeYvtCA64xGfUviwS,Watching Movies with the Sound Off,2013-06-18,16,https://i.scdn.co/image/ab67616d0000b27331b22e...,album,4LLpKhyESsyAXpc4laK94U,Mac Miller,
1,3P0mZWUWThSofCgvC9ZJYH,Avian,196756,50,2,0Wf65emw9eAwbjJ45gMMqp,Watching Movies with the Sound Off (10th Anniv...,2023-06-23,20,https://i.scdn.co/image/ab67616d0000b273881358...,album,4LLpKhyESsyAXpc4laK94U,Mac Miller,
144,4uPGQVcaiUGnNgEDYp0aol,Eggs Aisle,160066,33,10,1aDhas2rvJmiGw3zVVI3mm,Live From Space,2013-12-17,14,https://i.scdn.co/image/ab67616d0000b27331541d...,album,4LLpKhyESsyAXpc4laK94U,Mac Miller,


### Load

Creating accesses to upload the dataframe to google big query

In [ ]:
gbq_key = json.loads(os.environ.get("GBQ_KEY_PERSONAL"))
credentials = service_account.Credentials.from_service_account_info(gbq_key)

To avoid problems, we clear the name before uploading.

In [ ]:
artist = df['artist_name'][0].lower()
artist = re.sub('\s','_', artist)
artist = re.sub('[\.,-]','', artist)

I create 2 datasets in big query one to store the artist name and another to feed the dashboard.

In [ ]:
df.to_gbq(
    destination_table=f'spotify.{artist}',
    project_id='general-project-352815',
    if_exists='replace',
    credentials=credentials
    )

100%|██████████| 1/1 [00:00<00:00, 12372.58it/s]


In [ ]:
 df.to_gbq(
    destination_table=f'spotify.dashboard',
    project_id='general-project-352815',
    if_exists='replace',
    credentials=credentials
    )

100%|██████████| 1/1 [00:00<00:00, 19328.59it/s]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=39e57b38-a904-448a-a82a-d21973fb598d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>